# Transformers: El Corazón de la IA Moderna (AI Student Collective Madrid)
*Lauren Gallego Ropero -- 02/12/2025*

En este cuaderno se pueden encontrar todos los ejercicios prácticos que corresponden al taller. 
Se trata de cuatro ejemplos de uso de grandes modelos (todos basados en la arquitectura de Transformers) para tareas diferentes. 
El objetivo es mostrar la versatilidad de estos modelos, además de lo sencillo que es utilizar muchos modelos open-source con la 
librería **Transformers** de **HuggingFace**.

In [ ]:
import os
import math
import torch
import matplotlib.pyplot as plt
from PIL import Image
from datasets import load_dataset
from sklearn.decomposition import PCA

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    AutoFeatureExtractor,
    AutoModelForImageClassification,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    CLIPProcessor,
    CLIPModel,
    pipeline
)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

## 1. Embeddings
En esta primera tarea usaremos los embeddings preentrenados de uno de los primeros y más influyentes modelos transformer, BERT. 
Para ello tokenizaremos una frase, pasaremos cada token por el modelo y visualizaremos la representación de cada uno en dos dimensiones.

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


text = "The quick brown fox jumps over the lazy dog while the cat sleeps quietly on the sofa."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    embeddings = model.embeddings(inputs["input_ids"])

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
for token, embedding in zip(tokens, embeddings[0]):
    print(f"{token:12s} -> {embedding[:5]} ...")

In [ ]:
# Pasamos los embeddings a CPU 
emb_np = embeddings[0].detach().numpy() 

pca = PCA(n_components=2)
emb_2d = pca.fit_transform(emb_np)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(emb_2d[:,0], emb_2d[:,1])

for i, token in enumerate(tokens):
    plt.text(emb_2d[i,0]+0.01, emb_2d[i,1]+0.01, token, fontsize=12)

plt.title("Token embeddings visualized with PCA")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

Al final de cada ejercicio, eliminaremos el modelo y el tokenizer para limpiar el espacio en la GPU antes de guardar un nuevo modelo.

In [ ]:
del model
del tokenizer

torch.cuda.empty_cache()

## 2. Clasificación de texto
### 2.1 Zero-shot 
En esta tarea utilizaremos un modelo de lenguaje encoder-decoder para 'zero-shot classification'. Esto significa que, dado un texto, 
el modelo lo clasificará entre una serie de categorías dadas. Lo peculiar es que el modelo no ha sido entrenado para esta tarea
y nunca ha visto un ejemplo de ella, simplemente usa su conocimiento general.

In [ ]:
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, device_map="auto")

# Utilizamos la función pipeline de Transformers
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer)

Introducimos nuestro texto y nuestras posibles categorías

In [ ]:
text = "In 'Breaking Bad', Walter White, a high school chemistry teacher, turns to cooking methamphetamine after being diagnosed with terminal cancer."
candidate_labels = ["crime", "love", "fiction"]


result = classifier(text, candidate_labels)

# Visualización
labels = result['labels']
scores = result['scores']

plt.figure(figsize=(8,5))
plt.bar(labels, scores, color='skyblue')
plt.ylim(0, 1)
plt.ylabel("Probabilidad")
plt.title("Resultados")
plt.show()

### 2.2 Sentiment analysis

Para la siguiente tarea, también clasificaremos texto, pero en esta ocasión utilizando un modelo que ha sido entrenado para una tarea en específico (datos específicos).
En este caso haremos **sentiment analysis**, una de las tareas más populares por sus numerosas aplicaciones y la gran cantidad de datos disponibles online.

In [ ]:
dataset = load_dataset("tweet_eval", "sentiment", split="train[:50]")

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, device_map="auto")

sentiment_classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
model_labels = {0: "negative", 1: "neutral", 2: "positive"}


In [ ]:
texts = [str(t) for t in dataset["text"]]
results = sentiment_classifier(texts, batch_size=16)  

# Pasamos la respuesta a texto
predictions = []
for res in results:
    label_id = int(res["label"].split("_")[-1])  
    predictions.append(model_labels[label_id])
    
true_labels = [model_labels[l] for l in dataset["label"]]

# Calculamos accuracy
accuracy = sum([p == t for p, t in zip(predictions, true_labels)]) / len(true_labels)
print(f"Accuracy en los primeros 50 tweets: {accuracy*100:.2f}%")

In [ ]:
print("\nAlgunos Ejemplos:")
for i in range(5):
    print(f"Tweet: {dataset[i]['text']}")
    print(f"Valor real: {true_labels[i]}, Predicción: {predictions[i]}\n")

In [ ]:
del model
del tokenizer

torch.cuda.empty_cache()

## 3. Generación de texto
Para la siguiente tarea utilizaremos un modelo de lenguaje mucho más grande, el Mistral-7B-Instruct-v0.2
Necesitaremos un modelo de lenguaje (decoder only) diseñado para la generación autoregresiva, es decir, predecir constantemente la siguiente palabra / token.

Seguro que ya estás familiarizado con el formato de esta tarea: el modelo recibe un prompt de texto y nos devuelve una secuencia de texto.

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

def run(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
text = """The Office" is an American mockumentary sitcom that depicts the everyday lives of office employees working at Dunder Mifflin Paper Company in Scranton, Pennsylvania. 
          The show features a documentary-style filming with talking-head interviews from the staff. The series is known for its awkward humor, quirky characters, 
          and the often hilarious management style of regional manager Michael Scott. Key characters include Jim Halpert, Pam Beesly, Dwight Schrute, and many others who navigate 
          work, friendships, and office antics over the course of the series."""

question = "Who is the regional manager of the Scranton branch in The Office?"


prompt = f"""
Based on the following text, answer the question.

Text: "{text}"

Question: "{question}"

Answer:
"""

run(prompt)


In [ ]:
del model
del tokenizer

torch.cuda.empty_cache()

## 4. Clasificación de imágenes
Para esta última tarea necesitaremos un tipo de Transformer especial, capaz de procesar imágenes en lugar de text: el ViT o Vision Transformer.
Se trata de un modelo muy similar al Transformer tradicional, simplemente modificando el procesamiento de los datos de entrada (cada pixel corresponde con un token).

In [ ]:
model_name = "nateraw/vit-base-patch16-224-cifar10"
model = AutoModelForImageClassification.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model.eval()

In [ ]:
labels = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
cifar = load_dataset("cifar10", split="train").shuffle(seed=33).select(range(200))

In [ ]:
num_images = 9

# Imágenes y ground thruth
images = [cifar[i]["img"].resize((224,224)) for i in range(num_images)]
gts = [cifar[i]["label"] for i in range(num_images)]


# Pasamos por el modelo
inputs = feature_extractor(images=images, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

topk = torch.topk(torch.nn.functional.softmax(logits, dim=1), k=3, dim=1)

# Plot
cols = 3
rows = math.ceil(num_images / cols)
plt.figure(figsize=(15,15))
for i in range(num_images):
    ax = plt.subplot(rows, cols, i+1)
    plt.imshow(images[i])
    plt.axis("off")
    pred_text = "\n".join(
        f"{labels[idx.item()]} ({score.item():.2f})"
        for score, idx in zip(topk.values[i], topk.indices[i])
    )
    ax.set_title(f"GT: {labels[gts[i]]}\n{pred_text}", fontsize=9)

plt.tight_layout()
plt.show()